<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 11 - Experimentos</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
    * [1.1. Carga de datos](#section11)
    * [1.2. Definición de funciones auxiliares](#section12)
* [2. Experimentos](#section2)
    * [2.1. Recomendaciones sobre canciones conocidas](#section21)
    * [2.2. Predicciones sobre playlists seleccionadas](#section22)
    * [2.3. Predicciones sobre playlists aleatorias](#section23)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os
import numpy as np
import joblib
import json
import pandas as pd
import time
import pickle
import random

from collections import defaultdict
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.cross_validation import random_train_test_split

c:\pythonenv\mcidaen\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Variables globales
NUM_THREADS = 8
SEED = 0

# Directorio empleado para guardar/leer los datos generados
DATA_PATH = 'MPD_CSV'
MODELS_PATH = "models"
EXPERIMENTS_PATH = "backup"

ARTISTS_PATH = os.path.join(DATA_PATH,'mpd.artists.csv')
TRACKS_PATH = os.path.join(DATA_PATH,'mpd.tracks.csv')
PLSINFO_PATH = os.path.join(DATA_PATH,'mpd.playlists-info.csv')
MODEL_PATH = os.path.join(MODELS_PATH,'lightfm_model.pkl')
MPD_LFM_FILE = os.path.join(DATA_PATH, "mpd_lightfm.pickle")

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Tras completarse el entrenamiento, vamos a realizar una serie de experimentos en los que comprobamos los resultados que produce nuestro modelo realizando predicciones a las playlists que tenemos en nuestro conjunto de entrenamiento. Una vez realizados dichos experimentos, vamos a emplear las 10.000 playlists incompletas, que habíamos clasificado previamente en 10 categorías, para ver el desempeño de nuestro modelo.


<br>

---

<br>

<a id="section11"></a>
### <font color="#92002A">1.1 - Carga de datos</font>

<br>

In [5]:
#DataSet en formato LightFM
with open(MPD_LFM_FILE, "rb") as read_file:
    mpd_lfm_dict = pickle.load(read_file)
    
#Cargamos el modelo
tfg_model = joblib.load(MODEL_PATH)

In [6]:
# Dataframea utilizados durante los experimentos
df_tracks = pd.read_csv(TRACKS_PATH, index_col=0)
df_artists = pd.read_csv(ARTISTS_PATH,index_col=0)
df_plsinfo = pd.read_csv(PLSINFO_PATH, index_col=0)

# Array de numpy con los títulos de las pistas
tracks_names = df_tracks.track_name.to_numpy()
artists_names = df_artists.artist_name.to_numpy()
pls_names = df_plsinfo.name.to_numpy()

---

<br>

<a id="section12"></a>
### <font color="#92002A">1.2 - Definición de funciones auxiliares</font>
<br>


In [7]:
# Obtiene de un modelo los items similares al indicado
# mediante la similaridad coseno
def similar_items(item_id, model, N=-1):
    """
    :param item_id: Item del que obtener similares.
    :param model: Modelo a emplear.
    :param N: (Opcional) Número de items similares, por defecto se devuelven todos.
    :return: Lista de tuplas en formato (PID,SCORE).
    """
    (item_biased, item_representations) = model.get_item_representations()
    
    # Cosine Similarity
    scores = item_representations.dot(item_representations[item_id])
    item_norms = np.linalg.norm(item_representations, axis=1)
    item_norms[item_norms == 0] = 1e-10    
    scores /= item_norms
    best = np.argpartition(scores, -N)[-N:]    
    return sorted(zip(best, scores[best] / item_norms[item_id]),
                  key=lambda x: -x[1])

# https://github.com/lyst/lightfm/issues/244

In [8]:
# Realiza la prediccion de pistas a una o varias playlists
def make_prediction(model, data, pls_pids, labels, u_features=None, N=10):
    """
    :param model: Modelo a emplear.
    :param data: Matriz de interacciones (Sparse Matrix).
    :param pls_pids: Usuarios de los que se desea obtener recomendaciones.
    :param labels : Etiquetas/nombres de los items
    :param u_features: Matriz con las características de las playlists (Matriz de expansión).
    :param N: (Opcional) Número de items a predecir similares, por defecto se devuelven 10.
    :return: Lista de diccionarios.
    """
    results_list = []

    #Numero de usuarios e items de la matriz
    n_pls, n_items = data.shape

    #Genera recomendación para cada usuario
    for pl_id in pls_pids:
        
        #Items que el usuario conoce
        known_positives = data.tocsr()[pl_id].indices
        
        #Items que nuestro modelo predice al usuario
        scores = model.predict(pl_id, np.arange(n_items), user_features=u_features, num_threads=NUM_THREADS)
        #Ordena los items por puntuación
        top_items = np.argsort(-scores)

        # Borramos los items conocidos de las recomendaciones
        sorter = np.argsort(top_items)
        removable_indices = sorter[np.searchsorted(top_items, known_positives, sorter=sorter)]
        top_items = np.delete(top_items,removable_indices)

        # Resultados
        result_dict = {'pl_pid': pl_id,
                       'known_items' : list(known_positives),
                       'known_items_labels' : list(labels[known_positives]),
                       'recommended_items' : list(top_items[:N]),
                       'recommended_items_labels' : list(labels[top_items[:N]])}
        
        results_list.append(result_dict)
        
    return results_list

In [9]:
# Realiza predicciones a la lista de usuarios indicada
# (Cross-Validation)
def make_prediction_cv(model, train_data, test_data, pls_pids, labels, u_features=None, N=0):
    """
    :param model: Modelo a emplear.
    :param train_data: Matriz de interacciones (Sparse Matrix).
    :param test_data: Matriz de interacciones (Sparse Matrix).
    :param pls_pids: Usuarios de los que se desea obtener recomendaciones.
    :param labels: Etiquetas/nombres de los items.
    :param u_features: Matriz de características de playlist (Sparse Matrix).
    :param N: (Opcional) Número de items a predecir similares, por defecto se devuelven 10.
    :return: Lista de diccionarios.
    """
    results_list = []

    #Numero de usuarios e items de la matriz
    n_pls, n_items = train_data.shape

    #Genera recomendación para cada usuario
    for pl_id in pls_pids:
        
        #Items que el usuario conoce
        known_positives = train_data.tocsr()[pl_id].indices
        #Items para validación
        val_items = test_data.tocsr()[pl_id].indices        
                
        if N==0: 
            N = len(val_items)
        
        #Items que nuestro modelo predice al usuario
        scores = model.predict(pl_id, np.arange(n_items), user_features=u_features, num_threads=NUM_THREADS)
        #Ordena los items por puntuación
        top_items = np.argsort(-scores)

        # Borramos los items conocidos de las recomendaciones
        sorter = np.argsort(top_items)
        removable_indices = sorter[np.searchsorted(top_items, known_positives, sorter=sorter)]
        top_items = np.delete(top_items,removable_indices)

        # Resultados
        result_dict = {'pl_pid': pl_id,
                       'known_items' : list(known_positives),
                       'known_items_labels' : list(labels[known_positives]),
                       'recommended_items' : list(top_items[:N]),
                       'recommended_items_labels' : list(labels[top_items[:N]]),
                       'validation_items' : list(val_items),
                       'validation_items_labels' : list(labels[val_items]) }
        
        results_list.append(result_dict)
        
    return results_list

In [10]:
# Función que empleamos para evaluar los resultados de
# una predicción mediante validación cruzada
def plstrs_rec_analysis(rec_items, val_items):
    """
    :param rec_items: Items que ha recomendado el modelo.
    :param val_items: Items pertenecientes al conjunto de validación.
    :return: Diccionario con los resultados de las métricas.
    """
    val_set = {x['track_pid'] for x in val_items}
    rec10_set = {x['track_pid'] for x in rec_items[:10]}
    rec25_set = {x['track_pid'] for x in rec_items[:25]}
    rec50_set =  {x['track_pid'] for x in rec_items[:50]}
    rec100_set = {x['track_pid'] for x in rec_items[:100]}
    
    analysis_results = dict()    
    analysis_results['top_10'] = {'common' : rec10_set.intersection(val_set),
                                  'score' : len(rec10_set.intersection(val_set))/len(val_set)}    
    analysis_results['top_25'] = {'common' : rec25_set.intersection(val_set),
                                  'score' : len(rec25_set.intersection(val_set))/len(val_set)}
    analysis_results['top_50'] = {'common' : rec50_set.intersection(val_set),
                                  'score' : len(rec50_set.intersection(val_set))/len(val_set)}
    analysis_results['top_100'] = {'common' : rec100_set.intersection(val_set),
                                   'score' : len(rec100_set.intersection(val_set))/len(val_set)}
    
    return analysis_results

In [11]:
# Función auxiliar que empleamos para mostrar los resultados 
# tras evaluar una predicción
def plstrs_analysis_printer(results):
    print("TOP 10:  {:6}".format(results['top_10']['score']))
    print("TOP 25:  {:6}".format(results['top_25']['score']))
    print("TOP 50:  {:6}".format(results['top_50']['score']))
    print("TOP 100: {:6}".format(results['top_100']['score']))

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Experimentos</font>
<br>

<a id="section21"></a>
### <font color="#B20033">2.1 - Recomendaciones sobre canciones conocidas</font>

<br>

En este experimento, hemos seleccionado 3 canciones que conocemos y en las que podemos ver si nuestro modelo nos recomienda canciones que son relacionadas con éstas. Las canciones que hemos seleccionado son las siguientes:

* "**Toxic**" de *Britney Spears*.
* "**Cool**" de *Alesso*.
* "**Six Feet Under**" de *The Weeknd*. 

<br>

In [12]:
# Método empleado para realizar el experimento 1
def experiment_1(tracks_pids, model, df_t, a_labels, n_recs=20):
    results_list = []
    
    for track_pid in tracks_pids:
        result = dict()
        result['track'] = {'track_pid' : track_pid,
                           'track_name' : df_t.loc[track_pid].track_name,
                           'track_artist' : a_labels[df_t.loc[track_pid].artist_pid]}
        
        rec_items = similar_items(track_pid, model, N=n_recs+1)[1:]
        items_list = []    
        
        for t_id, _ in rec_items:
            track = {'track_pid' : str(t_id),
                     'track_name' : df_t.loc[t_id].track_name,
                     'track_artist' : a_labels[df_t.loc[t_id].artist_pid]}
            items_list.append(track)
        
        result['recommended_tracks'] = items_list
        results_list.append(result)
        
    return results_list

In [13]:
# Método para imprimir los resultados del experimento 1
def experiment_1_printer(result,N=10):
    print("*** {} ({}) ***\n".format(result['track']['track_name'],result['track']['track_artist']))
    for track in result['recommended_tracks'][:N]:
        print("{:50}  {}".format(track['track_name'][:50],track['track_artist']))

In [14]:
# PIDs de pistas que queremos probar
test_pids = [239356, 143490, 31390]

experiment_file = os.path.join(EXPERIMENTS_PATH,'experiment_1_results.json')
if os.path.isfile(experiment_file):
    with open(experiment_file) as json_file:
        results_experiment_1 = json.load(json_file)
else:
    results_experiment_1 = experiment_1(test_pids, tfg_model, df_tracks, artists_names)
    with open(experiment_file, 'w') as json_file:
        json_file.write(json.dumps(results_experiment_1, indent=4))

In [15]:
experiment_1_printer(results_experiment_1[0])

*** Toxic (Britney Spears) ***

Toxic                                               Britney Spears
TiK ToK                                             Kesha
Womanizer                                           Britney Spears
Don't Stop The Music                                Rihanna
Hollaback Girl                                      Gwen Stefani
Hips Don't Lie                                      Shakira
Wannabe                                             Spice Girls
Crazy In Love (feat. Jay-Z)                         Beyoncé
Hollaback Girl                                      Gwen Stefani
SexyBack                                            Justin Timberlake


In [16]:
experiment_1_printer(results_experiment_1[1])

*** Cool (Alesso) ***

City Of Dreams - Radio Edit                         Dirty South
I Could Be The One (Avicii Vs. Nicky Romero) - Rad  Avicii
Reload - Radio Edit                                 Sebastian Ingrosso
Runaway (U & I)                                     Galantis
Under Control                                       Calvin Harris
Heroes (we could be)                                Alesso
Ten Feet Tall                                       Afrojack
If I Lose Myself - Alesso vs OneRepublic            OneRepublic
Don't Look Down                                     Martin Garrix
Red Lights                                          Tiësto


In [17]:
experiment_1_printer(results_experiment_1[2])

*** Six Feet Under (The Weeknd) ***

Reminder                                            The Weeknd
Party Monster                                       The Weeknd
All I Know                                          The Weeknd
Sidewalks                                           The Weeknd
Shameless                                           The Weeknd
Acquainted                                          The Weeknd
Often                                               The Weeknd
Love To Lay                                         The Weeknd
Ordinary Life                                       The Weeknd
Might Not                                           Belly


Para la primera pista que hemos empleado como ejemplo, vemos que las canciones que nuestro modelo ha recomendado resultan ser de artistas similares al de la canción consultada. En este caso se da la peculiaridad de que vuelve a aparecer la misma canción que hemos consultado en los resultados. Esto se debe a que una canción puede aparecer en álbumes distintos y en cada uno de ellos *Spotify* la almacena con un identificador distinto.

En el segundo caso, nuestro modelo nos ha devuelto una serie de canciones cuyos estilos musicales son similares al de la canción que hemos elegido.

Para la última canción que hemos consultado, nuestro modelo nos ha devuelto 8 canciones del mismo artista (en las que la mayoría corresponden al mismo álbum) y 2 canciones de artistas similares. Probablemente este caso se deba a que la canción empleada no resulte muy conocida y sólo aparezca en playlists en las que figuren el resto de canciones que corresponden al álbum del artista.


---

<br>

<a id="section22"></a>
### <font color="#B20033">2.2 - Predicciones sobre playlists seleccionadas</font>

<br>

Para este experimento hemos seleccionado 3 playlists que contienen música que nos resulta conocida. A diferencia del experimento anterior, hemos aplicado validación cruzada sobre las playlists, de tal manera que extraemos el 20% de las pistas que contiene para posteriormente ver en qué parte de las *k* primeras posiciones aparecen. La métrica que emplearemos en este caso es la precisión (*precision@k*, proporción de elementos recomendados en el conjunto *top-k* que son relevantes). En nuestro caso vamos a consultar la precisión cuando *k* es igual a 10, 25, 50 y 100. Las playlist que hemos elegido son las siguientes:

* “**Teen**” (PID 203663).
* "**Kisses For Breakfast**” (PID 98312).
* "**F.E.E.L. – Lucky Luke**” (PID 4569).

<br>

In [18]:
# Dividimos el conjunto en train y test, para validación cruzada
train_tracks, test_tracks =  random_train_test_split(mpd_lfm_dict['plstrs_interactions'], test_percentage=0.2, 
                                                     random_state=np.random.RandomState(SEED))

In [19]:
# Método empleado para realizar el experimento A4
def experiment_2(model, train, test, pls_ids, df_t, a_labels, pls_labels, u_features=None, n_pred=500):
    t_labels = df_t.track_name.to_numpy()
    results_experiment = make_prediction_cv(model, train, test, pls_ids, t_labels, 
                                            u_features=u_features, N=n_pred)
    results_list = []
    
    for r in results_experiment:
        result = dict()        
        result['pl_pid'] = str(r['pl_pid'])
        result['pl_name'] = pls_labels[r['pl_pid']]
        known_items = []
        for i in range(len(r['known_items'])):
            t_artist_pid = df_t.loc[r['known_items'][i]].artist_pid
            known_items.append({'track_pid' : str(r['known_items'][i]),
                                'track_name' : r['known_items_labels'][i],
                                'track_artist': a_labels[t_artist_pid]})
        result['known_tracks'] = known_items
        recommended_items = []
        for i in range(len(r['recommended_items'])):
            t_artist_pid = df_t.loc[r['recommended_items'][i]].artist_pid
            recommended_items.append({'track_pid' : str(r['recommended_items'][i]),
                                      'track_name' : r['recommended_items_labels'][i],
                                      'track_artist' : a_labels[t_artist_pid]})
        result['recommended_tracks'] = recommended_items
        
        validation_items = []
        for i in range(len(r['validation_items'])):
            t_artist_pid = df_t.loc[r['validation_items'][i]].artist_pid
            validation_items.append({'track_pid' : str(r['validation_items'][i]),
                                     'track_name' : r['validation_items_labels'][i],
                                     'track_artist' : a_labels[t_artist_pid]})
        result['validation_tracks'] = validation_items
        
        results_list.append(result)
        
    return results_list

In [20]:
# Función para imprimir los resultados del experimento A4
def experiment_2_printer(result,max_items=10):
    print("***PLAYLIST {} | {}***".format(result['pl_pid'],result['pl_name']))
    print("#Pistas conocidas#")
    known_tracks = [(x['track_name'],x['track_artist']) for x in result['known_tracks'][:max_items]]
    for tr in known_tracks:
        print("{:50}  {}".format(tr[0][:50],tr[1]))
    print("\n#Pistas recomendadas#")
    rec_tracks = [(x['track_name'],x['track_artist']) for x in result['recommended_tracks'][:max_items]]
    for tr in rec_tracks:
        print("{:50}  {}".format(tr[0][:50],tr[1]))
    print("\n#Pistas para validación#")
    val_tracks = [(x['track_name'],x['track_artist']) for x in result['validation_tracks'][:max_items]]
    for tr in val_tracks:
        print("{:50}  {}".format(tr[0][:50],tr[1]))

In [21]:
# PIDs de playlists que queremos probar
test_pids = [203663,98312,4569]

experiment_file = os.path.join(EXPERIMENTS_PATH,'experiment_2_results.json')
if os.path.isfile(experiment_file):
    with open(experiment_file) as json_file:
        results_experiment_2 = json.load(json_file)
else:
    results_experiment_2 = experiment_2(tfg_model, train_tracks, test_tracks, test_pids,
                                          df_tracks, artists_names, pls_names)
    with open(experiment_file, 'w') as json_file:
        json_file.write(json.dumps(results_experiment_2, indent=4))

In [22]:
experiment_2_printer(results_experiment_2[0], max_items=20)

***PLAYLIST 203663 | Teen***
#Pistas conocidas#
Wannabe                                             Spice Girls
2 Become 1                                          Spice Girls
Say You'll Be There                                 Spice Girls
Viva Forever                                        Spice Girls
Too Much                                            Spice Girls
...Baby One More Time                               Britney Spears
Sometimes                                           Britney Spears
Toxic                                               Britney Spears
Oops!...I Did It Again                              Britney Spears
I'm a Slave 4 U                                     Britney Spears
Genie in a Bottle                                   Christina Aguilera
Rock Your Body                                      Justin Timberlake
Cry Me a River                                      Justin Timberlake
CAN'T STOP THE FEELING! (Original Song from DreamW  Justin Timberlake
Mirrors         

In [23]:
plstrs_rec_analysis(results_experiment_2[0]['recommended_tracks'],
                    results_experiment_2[0]['validation_tracks'])

{'top_10': {'common': {'238006'}, 'score': 0.2},
 'top_25': {'common': {'238006', '732850'}, 'score': 0.4},
 'top_50': {'common': {'238006', '732850'}, 'score': 0.4},
 'top_100': {'common': {'238006', '732850'}, 'score': 0.4}}

In [24]:
experiment_2_printer(results_experiment_2[1])

***PLAYLIST 98312 | Kisses For Breakfast ***
#Pistas conocidas#
The Other Side (with MAX & Ty Dolla $ign)           MAX
We Don't Talk Anymore (feat. Selena Gomez)          Charlie Puth
I Like Me Better                                    Lauv
High Hopes                                          Panic! At The Disco
The Other Side                                      Jason Derulo
Worth It                                            Fifth Harmony
Work from Home (feat. Ty Dolla $ign)                Fifth Harmony
I Will Never Let You Down                           Rita Ora
Attention                                           Charlie Puth
Sit Still, Look Pretty                              Daya

#Pistas recomendadas#
Old Town Road - Remix                               Lil Nas X
Sunflower - Spider-Man: Into the Spider-Verse       Post Malone
Sucker                                              Jonas Brothers
I Don't Care (with Justin Bieber)                   Ed Sheeran
bad guy                    

In [25]:
plstrs_rec_analysis(results_experiment_2[1]['recommended_tracks'],
                    results_experiment_2[1]['validation_tracks'])

{'top_10': {'common': {'33365'}, 'score': 0.3333333333333333},
 'top_25': {'common': {'305678', '33365'}, 'score': 0.6666666666666666},
 'top_50': {'common': {'305678', '33365'}, 'score': 0.6666666666666666},
 'top_100': {'common': {'305678', '33365'}, 'score': 0.6666666666666666}}

In [26]:
experiment_2_printer(results_experiment_2[2])

***PLAYLIST 4569 | F.E.E.L. – Lucky Luke***
#Pistas conocidas#
The Hum                                             Dimitri Vegas & Like Mike
Virus (How About Now)                               Martin Garrix
Body Talk (feat. Julian Perretta)                   Dimitri Vegas & Like Mike
Hangover                                            Dynoro
Drop It                                             Various Artists
F.E.E.L.                                            Lucky Luke
Cooler Than Me                                      Lucky Luke
I Could Be The One (Avicii Vs. Nicky Romero) - Rad  Avicii
Crazy                                               TRFN
Drive                                               TRFN

#Pistas recomendadas#
In My Mind                                          Dynoro
Old Town Road - Remix                               Lil Nas X
Still Cold / Pathway Private                        Night Lovell
Look at Me Now                                      Brennan Savage
Spotlight    

In [27]:
plstrs_rec_analysis(results_experiment_2[2]['recommended_tracks'],
                    results_experiment_2[2]['validation_tracks'])

{'top_10': {'common': {'359624'}, 'score': 0.25},
 'top_25': {'common': {'359624'}, 'score': 0.25},
 'top_50': {'common': {'359624', '904974'}, 'score': 0.5},
 'top_100': {'common': {'359624', '904974'}, 'score': 0.5}}

<br>

Si estudiamos con detalle las recomendaciones que ofrece nuestro modelo para completar las playlists seleccionadas, vemos que existe relación entre el contenido de todas ellas y sus títulos.

---

<br>

<a id="section23"></a>
### <font color="#B20033">2.3 - Predicciones sobre playlists aleatorias</font>

<br>

Repetimos el experimento anterior empleando 3 listas aleatorias.

<br>

In [28]:
# Método empleado para realizar el experimento A5:
def experiment_3(model, train, test, df_t, a_labels, pls_names, rand_items=3, u_features=None, n_pred=500):
    random_pl_pids = []
    for i in range(rand_items):
        random_pl_pids.append(random.randrange(0,1000000))
        
    return experiment_2(model, train, test, random_pl_pids, df_t, 
                         a_labels, pls_names, u_features, n_pred=250)

In [29]:
experiment_file = os.path.join(EXPERIMENTS_PATH,'experiment_3_results.json')

if os.path.isfile(experiment_file):
    with open(experiment_file) as json_file:
        results_experiment_3 = json.load(json_file)
else:
    results_experiment_3 = experiment_3(tfg_model, train_tracks, test_tracks, 
                                          df_tracks, artists_names, pls_names)
    with open(experiment_file, 'w') as json_file:
        json_file.write(json.dumps(results_experiment_3, indent=4))

In [30]:
experiment_2_printer(results_experiment_3[0])

***PLAYLIST 18103 | 💬-.-- -summer 19***
#Pistas conocidas#
Best Part (feat. H.E.R.)                            Daniel Caesar
Better Now                                          Post Malone
Stay                                                Post Malone
Psycho (feat. Ty Dolla $ign)                        Post Malone
Same Drugs                                          Chance the Rapper
Wonderwall                                          Oasis
iSpy (feat. Lil Yachty)                             KYLE
In My Feelings                                      Drake
glisten (interlude)                                 Jeremy Zucker
comethru (with Bea Miller)                          Jeremy Zucker

#Pistas recomendadas#
Loving Is Easy                                      Rex Orange County
Shotgun                                             George Ezra
Sunflower - Spider-Man: Into the Spider-Verse       Post Malone
Sunflower                                           Rex Orange County
Banana Pancakes  

In [31]:
plstrs_rec_analysis(results_experiment_3[0]['recommended_tracks'],
                    results_experiment_3[0]['validation_tracks'])

{'top_10': {'common': set(), 'score': 0.0},
 'top_25': {'common': set(), 'score': 0.0},
 'top_50': {'common': {'104518', '442568'}, 'score': 0.16666666666666666},
 'top_100': {'common': {'104518', '442568'}, 'score': 0.16666666666666666}}

In [32]:
experiment_2_printer(results_experiment_3[1])

***PLAYLIST 668333 | blame it on the goose***
#Pistas conocidas#
rockstar                                            Post Malone
Congratulations                                     Post Malone
pick up the phone                                   Young Thug
Come Get Her                                        Rae Sremmurd
This Could Be Us                                    Rae Sremmurd
Pleazer                                             Tyga
Feel Me                                             Tyga
Unforgettable                                       French Montana
Loaded (Bonus Track) [feat. DJ Carnage]             G-Eazy
HUMBLE.                                             Kendrick Lamar

#Pistas recomendadas#
God's Plan                                          Drake
Congratulations                                     Post Malone
No Role Modelz                                      J. Cole
BUTTERFLY EFFECT                                    Travis Scott
goosebumps                           

In [33]:
plstrs_rec_analysis(results_experiment_3[1]['recommended_tracks'],
                    results_experiment_3[1]['validation_tracks'])

{'top_10': {'common': set(), 'score': 0.0},
 'top_25': {'common': set(), 'score': 0.0},
 'top_50': {'common': {'110636', '142562', '241082'},
  'score': 0.21428571428571427},
 'top_100': {'common': {'110636', '142562', '241082', '441732'},
  'score': 0.2857142857142857}}

In [34]:
experiment_2_printer(results_experiment_3[2])

***PLAYLIST 264337 | neptuno surf shop opening***
#Pistas conocidas#
Cannonball                                          The Breeders
Disorder - 2007 Remastered Version                  Joy Division
Charlie Don't Surf                                  The Clash
Spanish Bombs                                       The Clash
Brand New Cadillac                                  The Clash
Lost in the Supermarket                             The Clash
Rudie Can't Fail                                    The Clash
Hateful                                             The Clash
Talking Backwards                                   Real Estate
Fingertips                                          The Brian Jonestown Massacre

#Pistas recomendadas#
Where Is My Mind?                                   Pixies
Shuggie                                             Foxygen
The Less I Know The Better                          Tame Impala
Under the Sun                                       DIIV
White Rabbit         

In [35]:
plstrs_rec_analysis(results_experiment_3[2]['recommended_tracks'],
                    results_experiment_3[2]['validation_tracks'])

{'top_10': {'common': set(), 'score': 0.0},
 'top_25': {'common': {'240487', '2547791'}, 'score': 0.2857142857142857},
 'top_50': {'common': {'240487', '2547791'}, 'score': 0.2857142857142857},
 'top_100': {'common': {'240487', '2547791'}, 'score': 0.2857142857142857}}

<br>

Tras concluir los experimentos realizados hemos comprobado que los resultados esperados, tras las recomendaciones de nuestro modelo para el completado de playlists, son los esperados. En determinados casos las pistas que hemos estrado para validación no se encuentran en el *Top 20*, pero si miramos las recomendaciones ofrecidas podemos ver que las canciones ofrecidas se asemejan lo suficiente con el resto de contenido de las playlists y sus títulos. 

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>